In [4]:
ord('牛')

29275

In [5]:
chr(29275)

'牛'

In [7]:
type("hello world".encode("utf8"))

bytes

In [9]:
len("你好 世界".encode("utf8")), len("你好 世界")

(13, 5)

```chr(0)``` 返回'\x00'

In [10]:
chr(0)

'\x00'

打印 chr(0) 不可见

In [11]:
print(chr(0))

 


In [13]:
string = "this is a test" + chr(0) + "string"
string

'this is a test\x00string'

In [14]:
print(string)

this is a test string


In [15]:
test_str = "hello 跃洋"
test_str_utf8 = test_str.encode("utf8")
test_str_utf8

b'hello \xe8\xb7\x83\xe6\xb4\x8b'

In [16]:
list(test_str_utf8)

[104, 101, 108, 108, 111, 32, 232, 183, 131, 230, 180, 139]

In [17]:
test_str_utf8.decode("utf-8")

'hello 跃洋'

In [24]:
"".join([bytes([b]).decode("utf-8") for b in "hello world".encode("utf8")])

'hello world'

In [2]:
import mmap

with open("/Users/yueyangcheng/WorkSpace/cs336/lab1/data/TinyStoriesV2-GPT4-valid.txt", mode="r+", encoding="utf-8") as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_WRITE) as m:
        data = m.read(10)
        print(data)

b"u don't ha"


In [3]:
from typing import List, Tuple, Dict
from collections import defaultdict
import mmap
import random

def load_and_sample_data(file_path: str,
                         sample_size: int,
                         special_tokens: str = "<|endoftext|>"
                         ) -> str:
    try:
        with open(file_path, "r+", encoding="utf-8") as f:
            with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
                documents = []
                start = 0
                while start < len(mm):
                    end = mm.find(special_tokens.encode("utf-8"), start)
                    if end == -1:
                        end = len(mm)
                        doc = mm[start:].decode("utf-8").strip()
                        if doc:
                            documents.append(doc)
                        break;
                    doc = mm[start: end].decode("utf-8").strip()
                    if doc:
                        documents.append(doc)

                    start = end + len(special_tokens)
                if len(documents) > sample_size:
                    documents = random.sample(documents, sample_size)

                return special_tokens.join(documents)
    except Exception as e:
        raise IOError(f"Error opening {file_path}: {e}")

In [4]:
load_and_sample_data("/Users/yueyangcheng/WorkSpace/cs336/lab1/data/TinyStoriesV2-GPT4-valid.txt", sample_size=10)

'One day, a pretty bird sat in the yard. It could whistle a nice song. Tim, a little boy, heard the bird and ran to see it. He liked the bird\'s song and wanted to be friends.\n"Hi bird! Can you be my friend?" Tim asked. The bird looked at Tim and whistled a happy tune. Tim was so happy to have a new friend.\nBut then, something strange happened. The bird changed! It was not a bird anymore. It was a big, friendly dog. Tim was so surprised, but he was even more happy. Now, he had a dog friend who could whistle. They played in the yard all day, and Tim taught the dog more songs to whistle.<|endoftext|>One day, a dog named Max found a whistle in the park. He picked it up with his mouth and ran home. Max wanted to show his friend, Tom the cat, his new toy.\nMax showed the whistle to Tom and said, "Look, I found a whistle!" Tom was tired but wanted to play. So, they went outside to play with the whistle. Max dropped the whistle, and Tom blew on it. It made a loud sound. They laughed and pla

In [5]:
test_list = ["hello", "world", "cyy"]
"".join(test_list)

'helloworldcyy'

In [6]:
list("".join(test_list))

['h', 'e', 'l', 'l', 'o', 'w', 'o', 'r', 'l', 'd', 'c', 'y', 'y']

In [7]:
import regex

GPT2_PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

def gpt_bytes_to_unicode_local() -> Dict[int, str]:
    bs = list(range(33, 127)) + list(range(161, 173)) + list(range(174, 256))
    cs = bs[:]

    n = 0
    for i in range(256):
        if i not in bs:
            bs.append(i)
            cs.append(n + 256)
            n += 1
    return {b: chr(c) for b, c in zip(bs, cs)}

def pre_tokenizer_document(doc: str, bytes_to_unicode_map: Dict[int, str]) -> List[List[str]]:
    tokens = regex.findall(GPT2_PAT, doc, flags=regex.UNICODE)
    sequences = []
    for token in tokens:
        token_unicode = "".join([bytes_to_unicode_map[b] for b in token.encode("utf-8")])
        sequences.append(list(token_unicode))

    return sequences

bytes_to_unicode_map = gpt_bytes_to_unicode_local()

pre_tokenizer_document("hello, world cyyang", bytes_to_unicode_map)

[['h', 'e', 'l', 'l', 'o'],
 [','],
 ['Ġ', 'w', 'o', 'r', 'l', 'd'],
 ['Ġ', 'c', 'y', 'y', 'a', 'n', 'g']]

In [10]:
regex.findall(GPT2_PAT, "hello world , cyyang dafda")

['hello', ' world', ' ,', ' cyyang', ' dafda']

In [2]:
from tqdm import tqdm
import time

# 示例：一个简单的循环
for i in tqdm(range(100)):
    time.sleep(0.1)  # 模拟耗时操作

100%|██████████| 100/100 [00:10<00:00,  9.53it/s]
